## Environmental zones 2018 - version 1.0, June 2020 1km

https://sdi.eea.europa.eu/catalogue/srv/eng/catalog.search#/metadata/6ef007ab-1fcd-4c4f-bc96-14e8afbcb688

In [2]:
# Configure plots for inline use in Jupyter Notebook
%matplotlib inline

import datetime as dt

# Utilities
import boto3
import dateutil
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import os
import rasterio
from time import process_time
# Start the stopwatch / counter 
t1_start = process_time() 


from f01_ingestion_utils import *
# Sentinel Hub
from sentinelhub import (
    CRS,
    BBox,
    ByocCollection,
    ByocCollectionAdditionalData,
    ByocCollectionBand,
    ByocTile,
    DataCollection,
    DownloadFailedException,
    MimeType,
    SentinelHubBYOC,
    SentinelHubRequest,
    SHConfig,
    bbox_to_dimensions,
    os_utils,
)

config = SHConfig()
config.instance_id = os.environ.get("SH_INSTANCE_ID")
config.sh_client_id = os.environ.get("SH_CLIENT_ID")
config.sh_client_secret = os.environ.get("SH_CLIENT_SECRET")
config.aws_access_key_id = os.environ.get("username")
config.aws_secret_access_key = os.environ.get("password")
    
    
list_byoc_collections()

# Stop the stopwatch / counter
t1_stop = process_time()
print("Elapsed time:", t1_stop, t1_start) 
print("Elapsed time during the whole program in seconds:", t1_stop-t1_start)


Collection name: urban_audit_2021_city
Collection id:  20774d38-7b2e-43e9-b574-66fa2954da1c
-------------------
Collection name: UrbanAtlas2018_10m_raster_demo
Collection id:  55c49191-4afa-4384-956e-e9b746f259f3
-------------------
Collection name: nuts3_2021
Collection id:  81a17892-38ff-470a-ae46-b64313acd7b3
-------------------
Collection name: ImperviousnessDensity2018_10m_raster
Collection id:  94c0149f-131a-4c0c-8809-11aeab5da253
-------------------
Collection name: UrbanAtlas2012_10m_raster_test2
Collection id:  974d338a-3814-4b83-b2f8-ee57f93c87c3
-------------------
Collection name: urban_audit_2021_fua
Collection id:  b17f90da-1308-439e-b675-6e3a87f883de
-------------------
Collection name: UrbanAtlas2012_10m_raster_test
Collection id:  b5a24968-e26a-42f4-bab6-09a9d38af697
-------------------
Collection name: nuts3_2021
Collection id:  ed7c0bf4-e85f-498c-ad18-f3e83fb89a6f
-------------------
Collection name: UrbanAtlas2012_10m_raster
Collection id:  f0585df9-c1af-4da7-b01d-c

In [20]:
# Initialize SentinelHubBYOC class
#byoc = SentinelHubBYOC(config=config)
#col_delete = byoc.get_collection("f27dd2b0-dba3-407e-bbe9-39e2cf8d9c08")
#byoc.delete_collection(col_delete)

''

In [5]:
# Initialize SentinelHubBYOC class
# Start the stopwatch / counter 
t1_start = process_time() 
byoc = SentinelHubBYOC(config=config)

new_collection = ByocCollection(name="env_zones_1km", s3_bucket="hub-fairicube0")
#byoc.delete_collection(new_collection) # for the delte of the collection
created_collection = byoc.create_collection(new_collection)
my_collection = byoc.get_collection(created_collection["id"])
# my_collection

#https://sentinelhub-py.readthedocs.io/en/latest/examples/byoc_request.html#Create-new-collection
#Name: Name: Name: data/d005_env_zones/eea_r_3035_1_km_envzones_p_2018_v01_r00/
input_folder ="data/d005_env_zones/eea_r_3035_1_km_envzones_p_2018_v01_r00"

tile_list = [
                "eea_r_3035_1_km_env-zones_p_2018_v01_r00_1_1_(BAND).tif",
                "eea_r_3035_1_km_env-zones_p_2018_v01_r00_1_2_(BAND).tif",
                "eea_r_3035_1_km_env-zones_p_2018_v01_r00_1_3_(BAND).tif",
                "eea_r_3035_1_km_env-zones_p_2018_v01_r00_1_4_(BAND).tif",
                "eea_r_3035_1_km_env-zones_p_2018_v01_r00_1_5_(BAND).tif",
                "eea_r_3035_1_km_env-zones_p_2018_v01_r00_1_6_(BAND).tif",
                "eea_r_3035_1_km_env-zones_p_2018_v01_r00_1_7_(BAND).tif",
                "eea_r_3035_1_km_env-zones_p_2018_v01_r00_2_1_(BAND).tif",
                "eea_r_3035_1_km_env-zones_p_2018_v01_r00_2_2_(BAND).tif",
                "eea_r_3035_1_km_env-zones_p_2018_v01_r00_2_3_(BAND).tif",
                "eea_r_3035_1_km_env-zones_p_2018_v01_r00_2_4_(BAND).tif",
                "eea_r_3035_1_km_env-zones_p_2018_v01_r00_2_5_(BAND).tif",
                "eea_r_3035_1_km_env-zones_p_2018_v01_r00_2_6_(BAND).tif",
                "eea_r_3035_1_km_env-zones_p_2018_v01_r00_2_7_(BAND).tif",
                "eea_r_3035_1_km_env-zones_p_2018_v01_r00_3_1_(BAND).tif",
                "eea_r_3035_1_km_env-zones_p_2018_v01_r00_3_2_(BAND).tif",
                "eea_r_3035_1_km_env-zones_p_2018_v01_r00_3_3_(BAND).tif",
                "eea_r_3035_1_km_env-zones_p_2018_v01_r00_3_4_(BAND).tif",
                "eea_r_3035_1_km_env-zones_p_2018_v01_r00_3_5_(BAND).tif",
                "eea_r_3035_1_km_env-zones_p_2018_v01_r00_3_6_(BAND).tif",
                "eea_r_3035_1_km_env-zones_p_2018_v01_r00_3_7_(BAND).tif",
                "eea_r_3035_1_km_env-zones_p_2018_v01_r00_4_1_(BAND).tif",
                "eea_r_3035_1_km_env-zones_p_2018_v01_r00_4_2_(BAND).tif",
                "eea_r_3035_1_km_env-zones_p_2018_v01_r00_4_3_(BAND).tif",
                "eea_r_3035_1_km_env-zones_p_2018_v01_r00_4_4_(BAND).tif",
                "eea_r_3035_1_km_env-zones_p_2018_v01_r00_4_5_(BAND).tif",
                "eea_r_3035_1_km_env-zones_p_2018_v01_r00_4_6_(BAND).tif",
                "eea_r_3035_1_km_env-zones_p_2018_v01_r00_4_7_(BAND).tif",
                "eea_r_3035_1_km_env-zones_p_2018_v01_r00_5_1_(BAND).tif",
                "eea_r_3035_1_km_env-zones_p_2018_v01_r00_5_2_(BAND).tif",
                "eea_r_3035_1_km_env-zones_p_2018_v01_r00_5_3_(BAND).tif",
                "eea_r_3035_1_km_env-zones_p_2018_v01_r00_5_4_(BAND).tif",
                "eea_r_3035_1_km_env-zones_p_2018_v01_r00_5_5_(BAND).tif",
                "eea_r_3035_1_km_env-zones_p_2018_v01_r00_5_6_(BAND).tif",
                "eea_r_3035_1_km_env-zones_p_2018_v01_r00_5_7_(BAND).tif"

            ]

# ingest tiles
for tile in tile_list:
    path_link = input_folder+"/"+tile
    new_tile = ByocTile(path=path_link) 
    created_tile = byoc.create_tile(created_collection, new_tile)
    tile = byoc.get_tile(collection=created_collection["id"], tile=created_tile["id"])
    print(tile["status"])
# path_link = input_folder
# new_tile = ByocTile(path=path_link) 
# created_tile = byoc.create_tile(created_collection, new_tile)
# tile = byoc.get_tile(collection=created_collection["id"], tile=created_tile["id"])
# Stop the stopwatch / counter
t1_stop = process_time()
print("Elapsed time:", t1_stop, t1_start) 
print("Elapsed time during the whole program in seconds:", t1_stop-t1_start)

WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
Elapsed time: 7.918334928 5.492722825
Elapsed time during the whole program in seconds: 2.4256121029999997


In [7]:
#############################################################
#name_of_your_collection = name_of_your_collection
name_of_your_collection = "env_zones_1km"
#############################################################

print ("START....")
# Initialize SentinelHubBYOC class
byoc = SentinelHubBYOC(config=config)
print ("START--------------------------")
# list collections and tiles
# from: https://sentinelhub-py.readthedocs.io/en/latest/examples/byoc_request.html


collections_iterator = byoc.iter_collections(name_of_your_collection)
my_collections = list(collections_iterator)[0]


data_collection = DataCollection.define_byoc(my_collections["id"])
data_collection

#'UrbanAtlas2018_10m_raster_v1'
#Collection id:  27c95f8f-fb6b-46f2-8f84-2ce7761eeccd
#DataCollection.BYOC_27c95f8f-fb6b-46f2-8f84-2ce7761eeccd
print("Collection name:", my_collections["name"])
print("Collection id: ", my_collections["id"])
tiles = list(byoc.iter_tiles(my_collections))
for tile in tiles:
        print("Tile status: ", tile['status'])
        print("Tile created: ", tile['created'])
        

print ("END----------------------------------------------------------")



START....
START--------------------------
Collection name: env_zones_1km
Collection id:  d582483f-b487-4ec0-b7cc-64be39af74f1
Tile status:  INGESTED
Tile created:  2023-06-13T08:21:25.122035Z
Tile status:  INGESTED
Tile created:  2023-06-13T08:21:24.097181Z
Tile status:  INGESTED
Tile created:  2023-06-13T08:21:22.172357Z
Tile status:  INGESTED
Tile created:  2023-06-13T08:21:24.607730Z
Tile status:  INGESTED
Tile created:  2023-06-13T08:21:22.680898Z
Tile status:  INGESTED
Tile created:  2023-06-13T08:21:21.970599Z
Tile status:  INGESTED
Tile created:  2023-06-13T08:21:23.896097Z
Tile status:  INGESTED
Tile created:  2023-06-13T08:21:24.500311Z
Tile status:  INGESTED
Tile created:  2023-06-13T08:21:23.492935Z
Tile status:  INGESTED
Tile created:  2023-06-13T08:21:23.094183Z
Tile status:  INGESTED
Tile created:  2023-06-13T08:21:22.478914Z
Tile status:  INGESTED
Tile created:  2023-06-13T08:21:24.400619Z
Tile status:  INGESTED
Tile created:  2023-06-13T08:21:23.795540Z
Tile status:  IN

In [25]:
#byoc.delete_collection(my_collections)


#xx error hello world
##for collection in my_collections:   
 # byoc.delete_collection(collection)
#print ("end")

tiles = list(byoc.iter_tiles(collection))
for tile in tiles:
    print("Tile status: ", tile['status'])
    print("Tile created: ", tile['created'])
    if(tile['status'] == "FAILED"):
        print("Ingestion failed error: ", tile['additionalData'])
print("-------------------")

DownloadFailedException: Failed to download from:
https://services.sentinel-hub.com/api/v1/byoc/collections/f27dd2b0-dba3-407e-bbe9-39e2cf8d9c08/tiles?
with HTTPError:
404 Client Error: Not Found for url: https://services.sentinel-hub.com/api/v1/byoc/collections/f27dd2b0-dba3-407e-bbe9-39e2cf8d9c08/tiles
Server response: "{"status": 404, "reason": "Not Found", "message": "Collection f27dd2b0-dba3-407e-bbe9-39e2cf8d9c08 not found.", "code": "COMMON_NOT_FOUND"}"